In [28]:
import pandas as pd
from transformers import DistilBertModel
from transformers import DistilBertTokenizer
from torchvision import transforms

import torch
from torch import nn
from torchvision import models
from typing import Dict, List
import cv2
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
model = models.resnet50(pretrained=True)

/home/djankows/ute/reverse-image-search/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/djankows/ute/reverse-image-search/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
for parameter in model.parameters():
    print(len(parameter))

64
64
64
64
64
64
64
64
64
256
256
256
256
256
256
64
64
64
64
64
64
256
256
256
64
64
64
64
64
64
256
256
256
128
128
128
128
128
128
512
512
512
512
512
512
128
128
128
128
128
128
512
512
512
128
128
128
128
128
128
512
512
512
128
128
128
128
128
128
512
512
512
256
256
256
256
256
256
1024
1024
1024
1024
1024
1024
256
256
256
256
256
256
1024
1024
1024
256
256
256
256
256
256
1024
1024
1024
256
256
256
256
256
256
1024
1024
1024
256
256
256
256
256
256
1024
1024
1024
256
256
256
256
256
256
1024
1024
1024
512
512
512
512
512
512
2048
2048
2048
2048
2048
2048
512
512
512
512
512
512
2048
2048
2048
512
512
512
512
512
512
2048
2048
2048
1000
1000


In [19]:
class ResNetEncoder(nn.Module):
    def __init__(self, model: str = "resnet18") -> None:
        super(ResNetEncoder, self).__init__()
        if model == "resnet18":
            self.model = models.resnet18(pretrained=True)
        elif model == "resnet50":
            self.model = models.resnet50(pretrained=True)
        else:
            raise ValueError("Incorrect type of ResNet architecture.")
        self.feature_extractor = nn.Sequential(*list(self.model.children()))

    def forward(self, images):
        return self.feature_extractor(images)

In [20]:
model = ResNetEncoder()

/home/djankows/ute/reverse-image-search/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/djankows/ute/reverse-image-search/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
model

ResNetEncoder(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [17]:
[len(parameter) for parameter in model.parameters()]

[64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 1000,
 1000]

In [29]:
CAPTIONS_PATH = "../../data/captions.csv"
IMAGES_PATH = "../../data/Images/"

In [30]:
df = pd.read_csv(CAPTIONS_PATH)

In [12]:
class TextEncoder(nn.Module):
    def __init__(self):
        super(TextEncoder, self).__init__()
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')

    def forward(self, input_ids, attention_mask) -> torch.Tensor:
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output[0]
        embedding = hidden_state[:, 0]
        return embedding


In [34]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [29]:
text = "Not a funny project"
tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length = 128,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )


{'input_ids': [101, 2025, 1037, 6057, 2622, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [31]:
class EmbeddingDataset(Dataset):
    def __init__(self, captions: pd.DataFrame, tokenizer: DistilBertTokenizer, transform: transforms.Compose) -> None:
        """Class for processing input texts and images. Transform both into proper form and torch tensors.

        Args:
            captions (pd.DataFrame): Contains two columns: image filename and description of it
            tokenizer (DistilBertTokenizer): Tokenizer
        """
        self.images = captions['image'].tolist()
        self.captions = captions['caption'].tolist()
        self.tokenizer = tokenizer
        self.transform = transform

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        caption: str = self.captions[idx]
        tokens_caption: Dict[str, List[int]] = tokenizer.encode_plus(
            caption,
            None,
            add_special_tokens=True,
            max_length = 128,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )

        image = cv2.imread(IMAGES_PATH+self.images[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)

        ids = tokens_caption['input_ids']
        mask = tokens_caption['attention_mask']
        token_type_ids = tokens_caption['token_type_ids']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'image': image
        }        

    def __len__(self):
        return len(self.captions)


In [32]:
transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224,224)),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

In [35]:
dataset = EmbeddingDataset(df, tokenizer=tokenizer, transform=transform)

In [61]:
loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=0)

In [45]:
resnet = models.resnet18(pretrained=True)

/home/djankows/ute/reverse-image-search/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/djankows/ute/reverse-image-search/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [48]:
import timm

In [68]:
class ImageEncoder(nn.Module):
    """
    Encode images to a fixed size vector
    """

    def __init__(
        self, model_name="resnet18", pretrained=True, trainable=True
    ):
        super().__init__()
        self.model = timm.create_model(
            model_name, pretrained, num_classes=0, global_pool="avg"
        )
        for p in self.model.parameters():
            p.requires_grad = trainable

    def forward(self, x):
        return self.model(x)

In [83]:
class ResNetEncoder(nn.Module):
    def __init__(self, model: str = "resnet18") -> None:
        super(ResNetEncoder, self).__init__()
        if model == "resnet18":
            self.resnet = models.resnet18(pretrained=True)
        elif model == "resnet50":
            self.resnet = models.resnet50(pretrained=True)
        else:
            raise ValueError("Incorrect type of ResNet architecture.")
        self.model = torch.nn.Sequential(*(list(self.resnet.children())[:-1]))

    def forward(self, images):
        return self.model(images).squeeze()

In [91]:
class ResNetEncoder(nn.Module):
    def __init__(self, model: str = "resnet18") -> None:
        super(ResNetEncoder, self).__init__()
        if model == "resnet18":
            resnet = models.resnet18(pretrained=True)
        elif model == "resnet50":
            resnet = models.resnet50(pretrained=True)
        else:
            raise ValueError("Incorrect type of ResNet architecture.")
        self.model = torch.nn.Sequential(*(list(resnet.children())[:-1]))

    def forward(self, images):
        return self.model(images).squeeze()


In [92]:
encoder = ImageEncoder()
resnet = ResNetEncoder()

/home/djankows/ute/reverse-image-search/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/djankows/ute/reverse-image-search/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [93]:
[len(parameter) for parameter in resnet.parameters()]

[64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 256,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512]

In [94]:
for data in loader:
    # print(resnet(data['image']).squeeze().size())
    # print(encoder(data['image']).size())
    # print(encoder(data['image']))
    # print(resnet(data['image']))
    print(torch.equal(encoder(data['image']), resnet(data['image'])))

    # break

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


KeyboardInterrupt: 

In [40]:
resnet = models.resnet18(pretrained=True)

# Set the model to evaluation mode
resnet.eval()

# Define a function to extract image embeddings
def get_image_embeddings(image):
    # Convert the image to a tensor
    transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224,224)),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    image_tensor = transform(image)

    # Pass the image through the model
    embeddings = resnet(image_tensor)

    # Return the embeddings
    return embeddings.squeeze()

/home/djankows/ute/reverse-image-search/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/djankows/ute/reverse-image-search/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
test = cv2.imread("/home/djankows/ute/reverse-image-search/data/Images/667626_18933d713e.jpg")

In [27]:
get_image_embeddings(test)

ValueError: expected 4D input (got 3D input)